# **- Machine Learning -**
# _* Ensemble Techniques *_
## Hitters Data **:**

In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor

In [21]:
df = pd.read_csv('Hitters_Data.csv')
df = df.dropna()
df

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


+ **Aşağıdaki KOD Bloğunda Sırası ile :**
+ + Öncelikle **, 'y' (Target)** kolonunu LOGARİTMİK olarak atayalım **:**
+ + Sonra **Numerik Olmayan (Kategorik Olan) Feature** kolonlarını **'get_dummies( )'** fonksiyonu ile **parçalayalım** ve **'binary'** şekilde **Numerik hale** getirelim. Yani **'dummy değişken'** hale getirelim. Bu numerik hale gelmiş hallerini de aşağıda yarattığımız **'dummies'** nesnesine **atayalım :**
+ + Sonra da bu Kategorik Olan Feature kolonlarını ve ayrıca Target kolonunu Veri Setimizden **düşürelim** ve bu haliyle de **,** aşağıda yarattığımız **'X__'** nesnesine atayalım. Ve bu **'X__'** nesnesinin içindeki Feature kolonlarının tamamını **'float64' tipine dönüştürelim :**
+ + En son da bu **Numerik hale gelen (dummies)** kolonlarından bize **gerekli olanları seçelim** ve **'concat( )'** fonksiyonunu kullanarak **diğer ('X__')** Feature kolonları ile **Birleştirelim.** Bu işlemin sonucunu da aşağıda yarattığımız **'X'** nesnesine **atayalım :**

In [22]:
y = np.log(df.Salary)

dummies = pd.get_dummies(df[['League' , 'Division' , 'NewLeague']])

#   Drop the column with the independent variable (Salary), and columns for which we created dummy variables :
#   Bağımsız değişkeni (Maaş) ve dummy (kukla) değişkenler oluşturduğumuz sütunları düşürün :
X__ = df.drop(['Salary' , 'League' , 'Division' , 'NewLeague'] , axis = 1).astype('float64')

#   Define the feature set X. :
#   Feature kümesini tanımlayın :
X = pd.concat([X__ , dummies[['League_N', 'Division_W', 'NewLeague_N']]] , axis = 1)

+ Öncelikle aşağıda bir **'Regression'** uygulayacağız.
<br> Aynı zamanda **'Regularization'** da yapacağız yani CEZALANDIRMA da yapacağız.
<br> CEZALANDIRMA yapacağımız için de Veri Setini **'Standardize'** etmemiz gerekecek (**StandardScaler()**) **. :**
<br> **++++ NOTE ++++**
<br> Veri Setini **Standardize** ederken SADECE TRAIN Seti kullanmamız gerkiyor...
<br> **'X_train'** ve **'X_test'** Setlerini de **,** bu **Standardize edilmiş** TRAIN Set üzerinden oluşturacağız...

In [23]:
#   Split data into training and test sets :
#   Verileri TRAIN ve TEST setlerine ayırın :
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=1)

scaler = preprocessing.StandardScaler()

scaler.fit(X_train)

X_trainStandard = scaler.transform(X_train)
X_testStandard = scaler.transform(X_test)

**++++ CRUCIAL NOTE ++++**
<br> Her şeyden önce **,** TRAIN Seti direkt **'Standardize'** ETMEYİP **,** ÖNCE **'Validation'**'u **AYIRIP , sonra 'Standardize'** EDİYORUZ...
<br> Çünkü, ilk TRAIN Seti KOMPLE **'Standardize'** eder ve bu işlemden SONRA TRAIN Seti **'Validation'** Set için parçalarsak (*Yani **Standardize** etmeden önce **'Pipeline'** oluşturmaz isek **, 'Cross Validation'** DEFAULT olarak böyle yapar...*) model **,** **'Standardize'** ederken **'Validation'**'u da görmüş olur ve **yanlış - aldatıcı** bir model kurmuş oluruz...

``Bundan dolayı ; ÖNCE 'Validation' Set'i ayıracağız , SONRA 'Standardize' edeceğiz...``

In [24]:
from sklearn.pipeline import Pipeline

regressor = RandomForestRegressor(random_state=0)

#   Number of Trees in random forest :
n_estimators = [100 , 200 , 300 , 400 , 500 , 600]

#   Number of features at every 'split' :
max_features = [3 , 4 , 5 , 6 , 7]

#   Create grid :
params = {
 'Regressor__n_estimators': n_estimators,
 'Regressor__max_features': max_features,
 }

pipe = Pipeline([('scaler', preprocessing.StandardScaler()) , ('Regressor', regressor)])

#   Random search of parameters :
#   EN İYİ Parametrelerin RASTGELE Search edilmesi (aranması) :
clf_grid = GridSearchCV(estimator=pipe , param_grid=params, 
                                cv=5 , verbose=2 , scoring='neg_mean_squared_error' , n_jobs= -1)
#   Fit the model :
clf_grid.fit(X_train , y_train)
#   Print results :
print(clf_grid.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
{'Regressor__max_features': 3, 'Regressor__n_estimators': 300}


**Yukarıdaki KOD Bloğunda olanlar :**
+ **``'GridSearchCV'`` modülü SIRASI ile :**
+ + Bir kombinasyonu alıyor. (Mesela ilk önce yukarıdaki **'100'e 3'** kombinasyonu olsun)... **'GridSearchCV' ,** içerisindeki **'(cv=5)'** parametresinden dolayı **'5-Fold Cross Validation'** yapması gerektiğini biliyor... Yani öncelikle TRAIN Veri Setini **'5'** parçaya bölmesi gerektiğini biliyor... Ve RASTGELE bir parçasını **'Validation'** Set olarak BİR KENARA AYIRIYOR.
+ + Şimdi bu ayırma işleminden sonra **,** Parametreleri SET etmem lazım diyor. Peki neyin Parametrelerini SET etmem lazım diyor **? 'Pipe'**'nin Parametrelerini SET etmem lazım diyor (**estimator=pipe**)...
+ + **'Pipe'**'ye geliyor... Şimdi **'Validation'** Set ayrıldıktan sonra ki geriye kalan TRAIN Sette **; ilk seçilen Parametre Kombinasyonunu (100'e 3)** FIT etme işlemini başlatıyor...
+ + **'Pipe'**'ın **,** öncelikle **'scaler'** işlemini yapmasını istiyor. Yani **Standardizasyon** işlemini bu aşamada gerçekleştiriyor... SONRA da **'Regressor'** Tekniğini kullanmasını istiyor. **'Regressor'**'a baktığımızda da **RandomForestRegressor()** Tekniğini kullanacağımızı görüyoruz...
+ + Şimdi de **'n_estimators'** ve **'max_features'** parametrelerimizin olası değerlerini SET etmemiz gerekiyor **(param_grid=params)**...
<br> KOD Bloğunda bu parametrelerin başlarına **'Regressor__'** koymamızın sebebi de **:**
<br> Çünkü bir PIPELINE yolluyoruz **,** ama bu Parametrelerin kime ait olduğunu bilmiyoruz... Yani PIPELINE'nin içerisindeki **'scaler'**'e mi ait yoksa **'Regressor'**'a mı ait bilmiyoruz... Ve **Regressor**'a ait Parametreler olduğunu bilmek için de başlarına **'Regressor__'** yazıyoruz...
<br> Böyle yazınca da **, 'RandomForestRegressor'**'a ait olan Parametreleri değiştirmesi gerektiğini biliyor...
+ + **'GridSearchCV' ,** her bir **Kombinasyon** için **'5'** farklı Validation Seti ayırıp (**5_Fold**) **;** bu her ayrım için **,** bu işlemleri en baştan tekrar ediyor ve sonra **yeni Kombinasyona** geçiyor (Mesela **100'e 4**)...

+ + GRID'i FIT ederken de Orijinal **'X_train'** ve **'y_train'** Setleri kullanıyoruz...

#### **++++ CRUCIAL NOTE ++++**
Yukarıda **,** PIPELINE içerisinde hem **'scaler'** hem de **'Regressor'** Teknikleri olduğu için **;** Sözlük türünde oluşturduğumuz Parametrelerin isimlerine direkt **"'n_estimators'"** ve **"'max_features'"** yazamıyoruz... PIPELINE içindeki **hangi** Tekniğe ait ise **,** onun ismini de MUHAKKAK belirtmemiz gerekiyor...
<br> **"'Regressor__n_estimators'"** ve **"'Regressor__max_features'"** ... gibi ...

+ Şimdi de EN İYİ Parametreleri bulduktan sonra EN BAŞTAN Modelimizi FIT ediyoruz... Ama artık burada **Standardize edilmiş 'X' TRAIN** Seti kullanmamız lazım...
<br> Daha sonra da yine **, ona göre Standardize edilmiş 'X' TEST** Setinde **Tahminleme** yapacağız **:**

In [30]:
clf = RandomForestRegressor(random_state=0 , n_estimators=300 , max_features=3)

clf.fit(X_trainStandard , y_train)

y_pred = clf.predict(X_testStandard)

print('Mean Squared Error:' , mean_squared_error(y_test , y_pred))

Mean Squared Error: 0.15424688036594958


In [31]:
r2_score(y_test , y_pred)

0.8293631151955395

+ Şimdi de **bir sürü Teknik** kullanalım ve her bir Tekniğin de farklı parametrelerini kullanarak deneyelim **:**

**+--- NOTE ---> 'Pipeline( )'** içerisine muhakkak bir Teknik göndermemiz lazım **!** Aşağıda **'RandomForestRegressor()'** tekniğini gönderiyoruz...

In [34]:
#    Create a PIPELINE :
pipe = Pipeline([('scaler', preprocessing.StandardScaler()) , ('Regressor', RandomForestRegressor())])

#   Create space of candidate learning algorithms and their hyperparameters :
search_space = [ { 'Regressor': [Ridge()] , 'Regressor__alpha': np.logspace(-3, 1, 10) } ,
                 { 'Regressor': [Lasso(max_iter=10000)] , 'Regressor__alpha': np.logspace(-3, 1, 10) } ,
                 { 'Regressor': [KNeighborsRegressor()] , 'Regressor__n_neighbors':[2, 3, 4, 5, 6] } ,
                 { 'Regressor': [RandomForestRegressor(random_state=0)] ,
                                'Regressor__n_estimators': [100, 200, 300, 400, 500] ,
                                'Regressor__max_features': [3, 4, 5, 6, 7] } ]

#   Create grid search :
clf = GridSearchCV(pipe , search_space , cv=5 , verbose=0 , scoring='neg_mean_squared_error')
#   Fit grid search :
best_model = clf.fit(X_train , y_train)
#   View best model :
best_model.best_estimator_.get_params()['Regressor']

RandomForestRegressor(max_features=3, n_estimators=300, random_state=0)

**YUKARIDA :**
+ + **GridSearchCV** modülü ile **; Pipeline (pipe)**'yi **, 'search_space'** atamasının içerisinde gezdirerek **, Pipeline** içerisinde oluşturduğumuz **"'Regressor' : RandomForestRegressor()" Tekniğini** her REPEAT'de **, 'search_space'** atamasının içerisindeki **Teknikler** ile değiştirebiliyoruz...
+ + Yani **'search_space'** atamasının içerisindeki HER BİR **Tekniği , 'Pipeline'** içerisinde deneyebiliyoruz...

**+--- NOTE --->** Yukarıda gördüğümüz gibi **, 'search_space'** atamasının içerisindeki Tekniklerle birlikte Parametreleri de var...
<br> **+--- NOTE --->** Daha yukarıda yaptığımız işlemlerin her birini **;** şimdi de **,** her bir Tekniği AYRI AYRI kullanarak yapacak... Ve En ideal Tekniği VE o Tekniğin En İyi Parametrelerini bize döndürecek...

+ Yukarıda gördüğümüz gibi **,** En İyi Tekniğimiz **'RandomForestRegressor'** imiş. Bu Tekniğin En İyi Parametreleri de **'max_features=3'** ve  **'n_estimators=300'** imiş... **:**

In [35]:
clf = RandomForestRegressor(random_state=0 , n_estimators=300 , max_features=3)

#   Train the model using the training sets : 
#   Modeli , TRAINING Setlerini kullanarak eğitin : ( y_pred = clf.predict(X_test) )

clf.fit(X_trainStandard , y_train)

y_pred = clf.predict(X_testStandard)

print('Mean Squared Error:' , mean_squared_error(y_test , y_pred))

Mean Squared Error: 0.15424688036594958


**++++ CRITICAL NOTE ++++**
+ Yukarıdaki Son 2 KOD Bloğu ile **;** Şimdiye kadar öğrendiğimiz hemen hemen her şeyi kullanmış olduk...
+ ANCAK **!** Ne kadar çok Tekniği bir arada denersek **,** yine OVERFITTING ihtimalini arttırmış oluruz **...!**
+ Aynı zamanda TESADÜFİ olarak bir Tekniğin herhangi bir Parametresi **,** Veri Setine çok iyi denk gelmiş olabilir yani çok iyi oturmuş olabilir ve bizi yine bu şekilde aldatabilir... Böyle olduğu için de **,** GERÇEK Hayatta denediğimizde bizi hayal kırıklığına uğratabilir...
+ Yani daha çok Teknik ve Parametre denedikçe **, 'Type-1' Error**'u arttırabiliriz...